In [ ]:
#Cell 1
#Imports
from pathlib import Path
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
sns.set(style="whitegrid")  # GraphPad-ish background


In [ ]:
# Cell 2
# --- Excel File Finder ---
DATA_DIR = Path(r"C:\Users\M298134\Desktop\AutoFlow\Tyler's Work")  # <-- change this if needed

def get_latest_excel_file(data_dir: Path) -> Path | None:
    excel_files = list(data_dir.glob("*.xlsx")) + list(data_dir.glob("*.xls"))
    if not excel_files:
        return None
    # sort by modification time (newest last)
    excel_files.sort(key=lambda f: f.stat().st_mtime)
    return excel_files[-1]

latest_file = get_latest_excel_file(DATA_DIR)

if latest_file is None:
    raise FileNotFoundError(f"No Excel files found in {DATA_DIR}")
else:
    print(f"Using file: {latest_file}")



In [ ]:
# Cell 3 
# Read Table
df_raw = pd.read_excel(latest_file)
display(df_raw.head())

print("Columns:", df_raw.columns.tolist())  # handy to see once

def extract_group_from_sample(sample_name: str) -> str | None:
    if not isinstance(sample_name, str):
        return None
    m = re.match(r"([A-Za-z]+)\d+", sample_name.strip())
    return m.group(1) if m else None

SAMPLE_COL = "Sample:"  # <-- the actual column name from your file

if SAMPLE_COL not in df_raw.columns:
    raise KeyError(
        f"Expected a sample ID column named '{SAMPLE_COL}' in the Excel file. "
        f"Found: {df_raw.columns.tolist()}"
    )

df = df_raw.copy()
df["Group"] = df[SAMPLE_COL].apply(extract_group_from_sample)

df[[SAMPLE_COL, "Group"]].head()



In [ ]:
# Cell 4 – Smart FlowJo column renaming

def clean_marker_name(m: str) -> str:
    """
    Light cleanup of marker names.
    You can expand this as you see patterns (e.g. CD8a+ -> CD8+).
    """
    # Simple tweaks that are common:
    m = m.replace("CD8a", "CD8")
    m = m.replace("CD4 ", "CD4")  # remove stray spaces
    return m

def make_nice_flowjo_name(col: str) -> str:
    """
    Turn a long FlowJo column name into something like:
    'VP2+ CD8+ Freq. of Parent'
    """
    # Keep some special columns as-is
    if col in ["Sample:", "Group"]:
        return col.replace(":", "")  # 'Sample:' -> 'Sample'
    
    # If there's no '|' separator, just return the column
    if "|" not in col:
        return col

    gate_part, metric_part = col.split("|", 1)
    gate_part = gate_part.strip()
    metric_part = metric_part.strip()  # e.g. 'Freq. of Parent'

    # Find all tokens that look like markers ending in + or -
    # e.g. CD8a+, VP2+, CD4-, PD-1+
    markers = re.findall(r"([A-Za-z0-9\-]+[+\-])", gate_part)

    # Clean up marker names a bit
    markers = [clean_marker_name(m) for m in markers]

    # If we got at least two markers, use the last two
    if len(markers) >= 2:
        m1 = markers[-1]      # last (e.g. VP2+)
        m2 = markers[-2]      # second to last (e.g. CD8+)
        nice = f"{m1} {m2} {metric_part}"
    elif len(markers) == 1:
        m1 = markers[0]
        nice = f"{m1} {metric_part}"
    else:
        # Fallback: just use the metric if no markers were detected
        nice = metric_part

    return nice

def shorten_column_names(columns):
    return {c: make_nice_flowjo_name(c) for c in columns}

# Apply to your df (which already has Group)
df_renamed = df.rename(columns=shorten_column_names(df.columns))

print("Old -> New column names (first 10):")
for old, new in list(shorten_column_names(df.columns).items())[:10]:
    print(f"{old!r} -> {new!r}")

df_renamed.head()


In [ ]:
# Cell 5
# Summarize by group

# Only keep numeric columns for stats
numeric_cols = df_renamed.select_dtypes(include="number").columns

group_summary = (
    df_renamed
    .groupby("Group")[numeric_cols]
    .agg(["mean", "std", "median", "count"])
)

group_summary


In [ ]:
# Cell 9 – Robust Stats Analysis (Outliers, Normality, Variance Tests, Correct Stats Choice)

import numpy as np
from scipy.stats import shapiro, levene, bartlett, f_oneway, kruskal
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests

def detect_outliers_grubbs(values, alpha=0.05):
    """
    Apply Grubbs' test for one outlier at a time.
    Returns indices of outliers.
    Requires n >= 3.
    """
    vals = values.copy().astype(float)
    outliers = []
    while len(vals) >= 3:
        mean = np.mean(vals)
        std = np.std(vals, ddof=1)
        if std == 0:
            break
        G = np.max(np.abs(vals - mean)) / std

        # Critical value for Grubbs (two-sided)
        n = len(vals)
        t_crit = stats.t.ppf(1 - alpha/(2*n), n - 2)
        G_crit = ((n - 1) / np.sqrt(n)) * np.sqrt(t_crit**2 / (n - 2 + t_crit**2))

        if G > G_crit:
            outlier_value = vals[np.argmax(np.abs(vals - mean))]
            outliers.append(outlier_value)
            vals = vals[vals != outlier_value]
        else:
            break

    return outliers


def variable_stats(df, variable, alpha=0.05):
    """
    Perform:
      - Outlier detection
      - Normality tests
      - Variance equality tests
      - Appropriate group comparison (ANOVA, Welch, or Kruskal)
    """
    result = {"variable": variable}

    # ----------- Gather values per group -----------
    groups = sorted(df["Group"].dropna().unique())
    group_vals = {g: df.loc[df["Group"] == g, variable].dropna().values for g in groups}

    # Must have ≥2 groups with data
    usable = [g for g in groups if len(group_vals[g]) >= 2]
    if len(usable) < 2:
        result.update({"test_used": "Insufficient data", "p": np.nan})
        return result

    # ----------- Outlier detection -----------
    outliers_removed = {}
    cleaned_vals = {}

    for g in usable:
        vals = group_vals[g]
        outs = detect_outliers_grubbs(vals, alpha=0.05)
        outliers_removed[g] = outs
        cleaned_vals[g] = np.array([v for v in vals if v not in outs])

    result["outliers_removed"] = outliers_removed

    # ----------- Normality tests (Shapiro-Wilk) -----------
    normal = []
    for g in usable:
        if len(cleaned_vals[g]) >= 3:
            p_norm = shapiro(cleaned_vals[g])[1]
            normal.append(p_norm > alpha)
        else:
            normal.append(False)

    result["normality_pass"] = all(normal)

    # ----------- Variance equality tests -----------
    val_list = [cleaned_vals[g] for g in usable]
    p_levene = levene(*val_list)[1]
    p_bartlett = bartlett(*val_list)[1]

    result["levene_equal_var"] = p_levene > alpha
    result["bartlett_equal_var"] = p_bartlett > alpha

    # ----------- Choose test -----------
    if result["normality_pass"]:
        if p_levene > alpha:
            test_name = "ANOVA"
            stat, p_value = f_oneway(*val_list)
        else:
            test_name = "Welch ANOVA"
            # SciPy does not have Welch ANOVA natively → do Kruskal fallback OR ping me for implementation
            stat, p_value = kruskal(*val_list)
    else:
        test_name = "Kruskal–Wallis"
        stat, p_value = kruskal(*val_list)

    result["test_used"] = test_name
    result["p"] = p_value

    return result


# ----------- Run stats for all variables -----------

stats_results = []
for var in numeric_cols:
    stats_results.append(variable_stats(df_renamed, var))

stats_df = pd.DataFrame(stats_results)

# Multiple-test correction (Benjamini–Hochberg FDR)
mask = stats_df["p"].notna()
reject, p_adj, *_ = multipletests(stats_df.loc[mask, "p"], alpha=0.05, method="fdr_bh")
stats_df.loc[mask, "p_adj"] = p_adj
stats_df.loc[mask, "significant_fdr<0.05"] = reject

# Sort by adjusted p-value
stats_df.sort_values("p_adj", inplace=True)

display(stats_df)


In [ ]:
# Cell 10 – GraphPad-style plots ONLY for significant variables

from itertools import combinations

alpha = 0.05

def get_sig_star(p):
    if p < 0.001:
        return '***'
    elif p < 0.01:
        return '**'
    elif p < 0.05:
        return '*'
    else:
        return 'ns'

sig_vars = anova_df.loc[anova_df["p"] < alpha, "variable"].tolist()

print(f"Significant variables (p < {alpha}):")
for v in sig_vars:
    print("  -", v)

if not sig_vars:
    print("No significant variables found.")
else:
    # choose a reference group for pairwise comparisons (like GraphPad)
    all_groups = sorted(df_renamed["Group"].dropna().unique())
    ref_group = all_groups[0]   # e.g. use 'A' as control; change if you want

    print(f"\nUsing '{ref_group}' as reference for pairwise tests.")

    for TARGET_VAR in sig_vars:
        print(f"\nPlotting significant variable: {TARGET_VAR}")
        
        # long-format data for seaborn
        plot_df = df_renamed[["Group", TARGET_VAR]].dropna().copy()
        plot_df = plot_df[plot_df["Group"].notna()]

        plt.figure(figsize=(6, 5))

        # Boxplot + swarm of individual points
        ax = sns.boxplot(
            data=plot_df,
            x="Group",
            y=TARGET_VAR,
            palette="Set2",
            showcaps=True,
            boxprops={"linewidth": 1.5},
            whiskerprops={"linewidth": 1.5},
            medianprops={"linewidth": 1.5},
        )
        sns.swarmplot(
            data=plot_df,
            x="Group",
            y=TARGET_VAR,
            color="0.25",
            size=5
        )

        # Basic labels
        plt.xlabel("Group")
        plt.ylabel(TARGET_VAR)
        overall_p = anova_df.loc[anova_df["variable"] == TARGET_VAR, "p"].values[0]
        plt.title(f"{TARGET_VAR}\nANOVA p = {overall_p:.3e}")

        # ---- Add pairwise significance vs reference group ----
        # we'll compare ref_group vs every other group
        x_positions = {g: i for i, g in enumerate(all_groups)}
        y_max = plot_df[TARGET_VAR].max()
        y_min = plot_df[TARGET_VAR].min()
        y_range = y_max - y_min if y_max > y_min else 1.0

        # start a bit above the max data
        current_height = y_max + 0.1 * y_range
        step = 0.08 * y_range  # vertical spacing between brackets

        for g in all_groups:
            if g == ref_group:
                continue

            vals_ref = plot_df.loc[plot_df["Group"] == ref_group, TARGET_VAR]
            vals_g   = plot_df.loc[plot_df["Group"] == g, TARGET_VAR]

            if len(vals_ref) < 2 or len(vals_g) < 2:
                continue  # skip tiny groups

            t, p_pair = stats.ttest_ind(vals_ref, vals_g, nan_policy="omit")
            star = get_sig_star(p_pair)

            if star == 'ns':
                continue  # don't draw NS comparisons

            x1 = x_positions[ref_group]
            x2 = x_positions[g]
            x_center = (x1 + x2) / 2

            # draw a bracket
            ax.plot(
                [x1, x1, x2, x2],
                [current_height, current_height + step/2, current_height + step/2, current_height],
                color="black",
                linewidth=1.2,
            )
            ax.text(
                x_center,
                current_height + step/2 + 0.02 * y_range,
                star,
                ha="center",
                va="bottom",
                fontsize=12,
            )

            current_height += step  # move up for next comparison

        plt.tight_layout()
        plt.show()


In [ ]:
# Cell 11 – Build GraphPad-style wide table (columns = groups, rows = replicates)

def make_graphpad_block(var: str) -> pd.DataFrame:
    """
    Given a column name `var` from df_renamed, returns a wide DataFrame:
      - columns = groups (A, B, C, ...)
      - rows = replicates (1, 2, 3, ...)
    """
    if var not in df_renamed.columns:
        raise KeyError(f"{var!r} is not a column in df_renamed. "
                       f"Available: {df_renamed.columns.tolist()}")

    sub = df_renamed[["Group", var]].dropna().copy()
    sub = sub[sub["Group"].notna()]

    # Add replicate index within each group
    sub["rep"] = sub.groupby("Group").cumcount()

    # Pivot: each group becomes its own column
    wide = sub.pivot(index="rep", columns="Group", values=var)

    # Order groups (A, B, C, …)
    wide = wide.reindex(sorted(wide.columns), axis=1)

    # Replicates start at 1 (nicer to look at)
    wide.index = wide.index + 1
    wide.index.name = None

    return wide




In [ ]:
# Cell 12 – Single-sheet Excel with labeled blocks for all significant variables

alpha = 0.05
sig_vars = anova_df.loc[anova_df["p"] < alpha, "variable"].tolist()

print(f"Found {len(sig_vars)} significant variables (p < {alpha}).")

if not sig_vars:
    print("No significant variables to export.")
else:
    blocks_for_concat = []

    for var in sig_vars:
        block = make_graphpad_block(var)  # rows = replicates, columns = groups (A, B, C…)

        # Title row: variable name in first cell, rest blank
        title_row = pd.DataFrame([[var] + [""] * (block.shape[1] - 1)],
                                 columns=block.columns)

        # Blank row between variables
        blank_row = pd.DataFrame([[""] * block.shape[1]],
                                 columns=block.columns)

        # Stack: title, table, blank
        combined = pd.concat([title_row, block, blank_row], ignore_index=True)
        blocks_for_concat.append(combined)

    # Stack all variables on top of each other
    all_sig_df = pd.concat(blocks_for_concat, ignore_index=True)

    output_path = DATA_DIR / "GraphPad_all_significant_blocks.xlsx"

    with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
        all_sig_df.to_excel(writer, sheet_name="All_significant", index=False)

    print(f"✅ Exported all significant variables into one file:\n  {output_path}")
    print("\nSheet 'All_significant' layout:")
    print("  • Title row = variable name")
    print("  • Next row = group labels (A, B, C, ...)")
    print("  • Following rows = replicates")
    print("  • Blank row between variables")
    print("\nTo use in GraphPad:")
    print("  1) Open the file")
    print("  2) Scroll to the marker you want")
    print("  3) Select its group table (title optional)")
    print("  4) Copy → Paste into GraphPad")


In [ ]:
# Cell 13 – Copy a single variable's table to clipboard for quick GraphPad paste

def copy_variable_to_clipboard(var: str):
    if var not in df_renamed.columns:
        raise KeyError(f"{var!r} is not a column in df_renamed. "
                       f"Available: {df_renamed.columns.tolist()}")
    
    block = make_graphpad_block(df_renamed, var)
    print(f"GraphPad-style table for {var}:")
    display(block)

    try:
        block.to_clipboard(index=False)
        print("\n✅ Table copied to clipboard! You can now paste directly into GraphPad.")
    except Exception as e:
        print("\n⚠️ Could not copy to clipboard from this environment.")
        print("You can still select the above table manually and copy it.")
        print("Error:", e)


In [ ]:
sig_vars = anova_df.loc[anova_df["p"] < 0.05, "variable"]
heat_df = (
    df_renamed
    .groupby("Group")[list(sig_vars)]
    .mean()
)

sns.heatmap(heat_df, annot=True)
